In [2]:
# Import all the libraries
import os
import datetime
import glob
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
import numpy as np
from whitebox.WBT.whitebox_tools import WhiteboxTools
from zipfile import ZipFile 
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import Affine,merge
from pathlib import Path 
import paramiko
import re
import shutil

# White toolbox integration
wbt = WhiteboxTools()

#import the Flood_Module_ALOS2
from Flood_Module_ALOS2 import ALOS2_pre_post


ALOS2_pre_post,ALOS2_post are two classes generated according to the data availability in the processing chain.

In [4]:
#Define a working directory
os.chdir("D:\\2020\\SL\\ALOS")
working_dir = os.getcwd()


In the working directory, create_project_dir function will create this following folder structure >>> 
1.Post_Data 
1.Pre_Data 
2.Result

In [5]:
# Since the pre and post data are available ALOS2_pre_post object is created here.
sftpp = ALOS2_pre_post(working_dir,project_name='SL')

In [6]:
sftpp.create_project_dir(working_dir)

In [7]:
## Listing data from the optemis system
# Input the activation name as shown in Optemis system and path to the key file.

#sftpp.list_ALOS_data(activation_name='20200704-Japan-Flood-Landslide-00374',sftp_key = 'F:/key/***')

In this case username, password, host, port and sftp key was provided accordingly in the Flood_Module_ALOS2 and it is shared only between selective members

If you dont have the access to the optemis system and you have the ALOS-2 Data in zip format you can put those zip files
accordingly in the Pre_Data and Post_Data folder in the working directory. 

In [ ]:
# Download the data from the system
#sftpp.download_ALOS_data(pre = True, post=False ,sftp_key =sftp_key = 'F:/key/***')

unzip_to_mosaic function will unzip ALOS-2 data when pre_mosaic, post_mosaic are given as true. More than one ALOS scene can be processed but they should be layed in consecutive frames.

In [8]:
# Unzip the ALOS data.

sftpp.unzip_to_mosaic(pre_mosaic = True,post_mosaic = True)

###########  ALOS_pre_post  ##########
All Post Files have been unzipped
###########  ALOS_pre_post  ##########
All Pre Files have been unzipped
###########  Raster Info  ##########
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 20309, 'height': 56604, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Geo-coded",GEOGCS["ITRF97 (geocentric)",DATUM["International_Terrestrial_Reference_Frame_1997",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6655"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4338"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(6.25, 0.0, 387641.69520179747,
       0.0, -6.25, 1092156.8469439559)}
########### 

Resampling of the unzipped images can be done using resample function. Define the pre and post image scale to change the 
resolution of the data. e.g., pre_scale = 2 --> reduce resolution by factor 2

In [9]:
sftpp.resample(pre = True ,post = True , pre_scale = 2, post_scale = 2)

###########  Raster Info  ##########
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 10154, 'height': 28310, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Geo-coded",GEOGCS["ITRF97 (geocentric)",DATUM["International_Terrestrial_Reference_Frame_1997",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6655"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4338"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",81],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(12.5, 0.0, 387611.25281723816,
       0.0, -12.5, 1092187.478577459)}
###########  Pre image resampled  ##########
###########  Raster Info  ##########
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 10154, 'heig

Pre and Post images are calibrated from the img_calibration function.

In [10]:
sftpp.img_calibration()

D:\2020\SL\ALOS/SL/2_Results\post_stage01.tif


C:\Users\Chatumal\scripts\Flood_Module_ALOS2_Compression.py:394: RuntimeWarning: invalid value encountered in less
  calib_post_intersect = np.where(calib_array < -30, -30, calib_array)


D:\2020\SL\ALOS/SL/2_Results\pre_stage01.tif
###########  PRE Post images calibrated  ##########


Apply speckle filtering for the images using speckle_filtering funtion by defining the filter size as shown below.

In [11]:
sftpp.speckle_filtering(filter_size = 5)

.\whitebox_tools.exe --run="LeeSigmaFilter" --input='D:\2020\SL\ALOS/SL/2_Results\post_stage01_calib.tif' --output='D:\2020\SL\ALOS/SL/2_Results\post_calib_spec.tif' --filterx=5 --filtery=5 --sigma=10.0 -m=5.0 -v

*****************************
* Welcome to LeeSigmaFilter *
*****************************
Reading data...
Saving data...
Output file written
Elapsed Time (excluding I/O): 5.823s
.\whitebox_tools.exe --run="LeeSigmaFilter" --input='D:\2020\SL\ALOS/SL/2_Results\pre_stage01_calib.tif' --output='D:\2020\SL\ALOS/SL/2_Results\pre_calib_spec.tif' --filterx=5 --filtery=5 --sigma=10.0 -m=5.0 -v

*****************************
* Welcome to LeeSigmaFilter *
*****************************
Reading data...
Saving data...
Output file written
Elapsed Time (excluding I/O): 5.858s
###########  Speckle Filtered  ##########


Change function will give the differnce image of the pre and post scenes.

In [12]:
sftpp.change()

###########  Change image is created  ##########


reproject_wgs84 function will transfrom all the speckle filtered images and change image in to WGS84 co-ordiante system.

In [13]:
sftpp.reproject_wgs84()

###########  Change image reprojected  ##########
###########  Speckle images reprojected  ##########


From thresholding function you can delinite the flood water by defining the threshold value.

In [14]:
sftpp.thresholding(threshold_value=-3)

C:\Users\Chatumal\scripts\Flood_Module_ALOS2_Compression.py:573: RuntimeWarning: invalid value encountered in less
  threshold = np.where(post_calib_spec < threshold_value,1,0)


###########  Threshold Applied  ##########


You can apply majority filtering on the threshold image by providing the filter size and give the threshold value 
accordingly to apply it on the exact threshold image.

In [15]:
sftpp.maj_filtering(filter_size = 5,threshold = -3)

.\whitebox_tools.exe --run="MajorityFilter" --input='D:\2020\SL\ALOS\SL\2_Results\change_threshold_-3.tif' --output='D:\2020\SL\ALOS/SL/2_Results/threshold_-3_maj_5.tif' --filterx=5 --filtery=5 -v

*****************************
* Welcome to MajorityFilter *
*****************************
Reading data...
Saving data...
Output file written
Elapsed Time (excluding I/O): 3.550s
###########  majotiry filtered  ##########


Apply ras2poly function to polygonize the final result.

In [16]:
sftpp.ras2poly(threshold =-3,filter_size = 5)

.\whitebox_tools.exe --run="RasterToVectorPolygons" --input='D:\2020\SL\ALOS\SL\2_Results\threshold_-3final_maj_5.tif' --output='D:\2020\SL\ALOS/SL/2_Results/Detected_Flood_Water.shp' -v

*************************************
* Welcome to RasterToVectorPolygons *
*************************************
Reading data...
Saving data...
Output file written
Elapsed Time (excluding I/O): 46.74s
###########  Polygonized  ##########


In [17]:
# This function will clean up the un-necessary files in the result folder.
sftpp.clean_result_folder()

###########  Floder Cleaned  ##########
